## Next steps:

- Investigate data
- Investigate Null values
- First analysis (GROUP BY, standard metrics)


## Ideas

- Interesting could be a cart rate. How much and what from the cart got purchased at the end. And how long did it stay in the  cart.

- Same from view to cart or purchase.

- These could be analysied by user, category or brand

## 1.0 Prerequisite and environment checks 

In [28]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import impala as ip
import prettytable as pt
from prettytable import from_db_cursor
from impala.dbapi import connect
from impala.util import as_pandas

In [29]:
#Showing graphs inline
%matplotlib inline

In [4]:
#Connection to the Imapla database
conn = connect('localhost', port=21050)
cursor = conn.cursor()

In [4]:
#First test query to show the table and the data types
cursor.execute('DESCRIBE sales.sales')
cursor.fetchall()

[('event_time', 'timestamp', ''),
 ('event_type', 'string', ''),
 ('product_id', 'bigint', ''),
 ('category_id', 'bigint', ''),
 ('category_code', 'string', ''),
 ('brand', 'string', ''),
 ('price', 'double', ''),
 ('user_id', 'bigint', ''),
 ('user_session', 'string', '')]

In [6]:
col_names = ['event_time', 'event_type', 'product_id', 'category_id','category_code','brand','price','user_id','user_session']


In [27]:
#First test query to the results
QUERY = 'SELECT * FROM sales.sales LIMIT 10'

In [28]:
cursor.execute(QUERY)
results = pd.DataFrame(cursor.fetchall())
results.columns = col_names

In [29]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   event_time     10 non-null     datetime64[ns]
 1   event_type     10 non-null     object        
 2   product_id     10 non-null     int64         
 3   category_id    10 non-null     int64         
 4   category_code  6 non-null      object        
 5   brand          9 non-null      object        
 6   price          10 non-null     float64       
 7   user_id        10 non-null     int64         
 8   user_session   10 non-null     object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 848.0+ bytes


In [30]:
results

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-28 17:27:50,view,54100012,2145153815500816706,None,matrix,36.13,515522739,1862607d-488b-49d6-8742-ebc6079b53cc
1,2019-11-28 17:27:50,view,5701166,2053013553970938175,auto.accessories.player,None,143.89,550006854,76ca3f45-1834-45b6-884c-1df9a0f0ba43
2,2019-11-28 17:27:50,view,100023426,2053013558920217191,computers.notebook,hp,771.96,512543257,a7888cc5-8eb3-4364-8007-4f3b154c9729
3,2019-11-28 17:27:50,view,1004833,2053013555631882655,electronics.smartphone,samsung,168.65,515053770,be2c2a66-de21-4cb8-9add-048e734522dd
4,2019-11-28 17:27:50,view,26200979,2053013563693335403,None,lucente,204.64,554063553,24ee2c73-a3dc-4adc-8293-0b24dd46fac6
5,2019-11-28 17:27:50,view,17600880,2053013558895051365,None,vichy,26.26,569742223,b1476955-c767-4c72-9e26-989640f51721
6,2019-11-28 17:27:50,view,11500123,2053013554625249641,None,xiaomi,30.86,518262204,3f2a7b9c-f180-4fe1-8e29-019e62906182
7,2019-11-28 17:27:50,view,1000978,2053013555631882655,electronics.smartphone,samsung,300.91,520157929,69e3bd80-38cd-410c-b729-474a2a0e3dbe
8,2019-11-28 17:27:50,view,1004433,2053013555631882655,electronics.smartphone,samsung,334.37,578385425,ac79eea1-c73d-4320-9d66-0ce4c61a1eef
9,2019-11-28 17:27:50,view,1005186,2053013555631882655,electronics.smartphone,samsung,771.94,517256643,5173a640-687a-492c-a095-88ded046d4f5


## 2.0 Analysing Null values

First step is to analyse where the Null values are and later see if thoses columns need to be dropped, modified or can be left like this. 

The first query prints the percentage of the the Null values in each column. 

In [32]:
QUERY = """SELECT
            COUNT(event_time)/COUNT(*)*100, 
            COUNT(event_type)/COUNT(*)*100,
            COUNT(product_id)/COUNT(*)*100,
            COUNT(category_id)/COUNT(*)*100,
            COUNT(category_code)/COUNT(*)*100,
            COUNT(brand)/COUNT(*)*100,
            COUNT(price)/COUNT(*)*100,
            COUNT(user_id)/COUNT(*)*100,
            COUNT(user_session)/COUNT(*)*100
            FROM sales.sales"""

In [35]:
cursor.execute(QUERY)
results = pd.DataFrame(cursor.fetchall())
results.columns = ['event_time', 'event_type', 'product_id', 'category_id','category_code','brand','price','user_id','user_session']
results

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,100.0,100.0,100.0,100.0,67.559216,86.3351,100.0,100.0,99.999985


It seems there are missing values for the category code (33%) eventhough there is each time a category id.
Maybe not all values are properly recorded?

The brand is blank for 14% of the cases. Maybe get the missing values based on the category code?

Also some values for the user_session are missing.

### 2.1 Missing values in category_code

Two views are created based on the sales dataset.
One view has only the NULL values as category code(cat_code_null) and the other only the none Null vlaues (cat_code_not_null).
Now a join is performed on category_id with both views. This verifies if not some of the missing values from category id could ne found in the data.

In [53]:
QUERY = """CREATE VIEW sales.cat_code_null AS 
            SELECT category_id, category_code, brand 
            FROM sales.sales 
            WHERE category_code IS NULL"""
cursor.execute(QUERY)

In [54]:
QUERY = """CREATE VIEW sales.cat_code_not_null AS
            SELECT category_id, category_code, brand
            FROM sales.sales
            WHERE category_code IS NOT NULL"""
cursor.execute(QUERY)

In [56]:
QUERY = """SELECT t1.category_id, t1.category_code, t2.category_code
            FROM cat_code_null t1 JOIN cat_code_not_null t2
            ON t1.category_id = t2.category_id LIMIT 10"""
cursor.execute(QUERY)
cursor.fetchall()

[]

The result indicate that we can not fill missing values like this.

Next the missing valies in category_code shall be inspected further if we can keep then or need to be dropped.

In [21]:
QUERY = """SELECT event_type, COUNT(*) FROM sales.sales WHERE category_code IS NULL GROUP BY event_type
"""
cursor.execute(QUERY)
results = pd.DataFrame(cursor.fetchall())


In [22]:
results

,0,1
0,cart,826493
1,purchase,234218
2,view,20837460


Most of the missing category values are from the event_type view. Where the others are relativly small. 
We can keep the missing values for category_id since we are more interested in the event_type cart and purchase.

The missing values are propably caused by not entering the code. It does not look like there is a certain relation between brand and the missing category code after having a look.

In [17]:
QUERY = """SELECT * FROM sales.sales WHERE category_code IS NULL AND event_type = 'purchase'
"""
cursor.execute(QUERY)
results = pd.DataFrame(cursor.fetchall())
results.columns = col_names

In [18]:
results

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-11-16 08:32:35,purchase,12719447,2053013553559896355,None,toway,262.04,540276994,3f44f715-d75b-434e-8ec3-7814510bb361
1,2019-11-16 08:32:35,purchase,4100126,2053013561218695907,None,sony,348.66,514219945,9b0507b0-7751-4d15-b8bb-4786cfc973a3
2,2019-11-16 08:32:38,purchase,12720607,2053013553559896355,None,joyroad,46.08,567327346,c4d694fa-207a-43d7-9bed-babe84c06d23
3,2019-11-16 08:32:38,purchase,12705001,2053013553559896355,None,cordiant,41.96,564262075,d7690897-3036-481e-8e9b-200c2cf4e24a
4,2019-11-16 08:32:41,purchase,2600248,2053013563970159485,None,hansa,330.23,568941627,3f8e517a-c416-4a82-abfd-aa985eff76ba
...,...,...,...,...,...,...,...,...,...
234213,2019-11-17 19:44:00,purchase,26400266,2053013563651392361,None,lucente,119.18,519191410,970f319b-1cfd-4cba-8ef7-e91e4b71bf27
234214,2019-11-17 19:44:01,purchase,12712237,2053013553559896355,None,goodyear,43.76,563881729,9016ee00-990e-447e-9dd5-88218f1cb78a
234215,2019-11-17 19:45:04,purchase,22700084,2053013556168753601,None,force,239.31,527418938,69d7142b-9f73-4d4a-b40d-30397ab6a154
234216,2019-11-17 19:45:44,purchase,26402784,2053013563651392361,None,None,97.30,522043731,83ba6a93-3cb3-4226-bf1d-efe4edb4ae86


In [24]:
QUERY = """SELECT brand, SUM(price) AS revenue FROM sales.sales
            WHERE category_code IS NULL AND event_type = 'purchase' GROUP BY brand
"""
cursor.execute(QUERY)
results = pd.DataFrame(cursor.fetchall())
results.columns = ['brand', 'revenue']
results

,brand,revenue
0,domyos,10718.37
1,wenge,926.39
2,giorgioarmani,3322.74
3,fubag,2423.77
4,nikon,186721.98
...,...,...
1691,fender,7877.51
1692,activision,1684.25
1693,sisley,406.18
1694,dr.hoffman,127.93


In [38]:
results.nlargest(10, 'revenue')

,brand,revenue
99,None,7272677.76
655,lucente,3527545.57
545,sony,2809534.50
1294,cordiant,787330.86
1077,bosch,556160.49
520,dauscher,545324.66
1318,nokian,438955.95
740,xiaomi,385816.73
822,canon,364787.42
880,navien,337943.07
